In [2]:
from mangaki.utils.algo import get_algo_backup, get_dataset_backup

svd = get_algo_backup('svd')

In [14]:
svd.M.shape  # Ratings (matrice approchée)

(2062, 10132)

In [13]:
svd.U.shape  # Users

(2062, 20)

In [15]:
svd.VT.shape  # Works (M = U x ∑ x VT)

(20, 10132)

In [16]:
import numpy as np

svd.predict(np.array([(1, 42), (2, 42)]))  # couples user_id, work_id

array([ 1.06063505,  1.85021549])

In [17]:
dataset = get_dataset_backup('svd')  # Pour avoir les titres

In [18]:
dataset.decode_work[42]

6194

In [23]:
Work.objects.get(id=6194)

<Work: Love Berrish!>

In [20]:
dataset.decode_user[1]

218

In [25]:
User.objects.get(id=218)

In [26]:
cham = User.objects.get(username='cham')

In [27]:
svd.M.shape

(2062, 10132)

In [28]:
from mangaki.utils.algo import fit_algo

In [29]:
from mangaki.utils.values import rating_values
rating_values

{'dislike': -2,
 'favorite': 4,
 'like': 2,
 'neutral': 0.1,
 'willsee': 0.5,
 'wontsee': -0.5}

In [30]:
triplets = Rating.objects.values_list('user_id', 'work_id', 'choice')
dataset, svd = fit_algo('svd', triplets)  # Train & save new SVD model

Computing M: (2062 × 10132)
Chrono: fill and center matrix [8q, 2928ms]
Shapes (2062, 20) (20,) (20, 10132)
Chrono: factor matrix [8q, 1626ms]


In [31]:
encoded_cham = dataset.encode_user[cham.id]

In [32]:
cham_line = svd.M[encoded_cham]

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

scores = cosine_similarity([cham_line], svd.M)[0]

In [34]:
scores.shape

(2062,)

In [35]:
np.argsort([10, 30, -5, 2])

array([2, 3, 0, 1])

In [36]:
np.sort([10, 30, -5, 2])

array([-5,  2, 10, 30])

In [37]:
neighbors = np.argsort(scores)[:20]

In [38]:
dataset.decode_users(neighbors)

[1683,
 1435,
 1764,
 1815,
 616,
 266,
 522,
 1011,
 91,
 1184,
 461,
 1918,
 1497,
 28,
 358,
 1900,
 119,
 2130,
 911,
 64]

In [39]:
true_neighbors = User.objects.in_bulk(dataset.decode_users(neighbors))

In [40]:
from mangaki.utils.knn import MangakiKNN

In [41]:
knn = MangakiKNN()

In [42]:
knn.NB_NEIGHBORS

20

In [43]:
knn.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)

In [44]:
knn.fit(dataset.anonymized.X, dataset.anonymized.y)

In [45]:
knn.M

<2062x10132 sparse matrix of type '<class 'numpy.float64'>'
	with 332124 stored elements in LInked List format>

In [47]:
knn.save(knn.get_backup_filename())  # Don't forget to replace defaultdict(lambda: 0) with Counter()